In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install PyPDF2 #necessary libraries for the module
!pip3 install nltk
!pip3 install autocorrect
!pip3 install transformers

In [3]:
import tensorflow as tf
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
import PyPDF2
#extracting the pdf file

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

In [5]:
#text pre-processing libraries

import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.corpus import stopwords, brown

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
stop_words  = stopwords.words('english')
from autocorrect import Speller
from nltk.tokenize import word_tokenize

#lower casing the text
def to_lower(text):

    """
    Converting text to lower case as in, converting "Hello" to  "hello" or "Hi" to "hi".
    """

    # Specll check the words
    spell  = Speller(lang='en')

    texts = spell(text)

    return ' '.join([w.lower() for w in word_tokenize(text)])

#cleaning the text
def clean_text(lower_case):
    # split text phrases into words
    words  = nltk.word_tokenize(lower_case)


    # Create a list of all the punctuations
    punctuations = [ '/', '!', '?', ';', ':', '(',')', '[',']', '-', '_', '%']

    # Remove all the special characters
    punctuations = re.sub(r'\W', ' ', str(lower_case))


    # Getting rid of all the words that contain numbers in them
    w_num = re.sub('\w*\d\w*', '', lower_case).strip()

    # remove all single characters
    lower_case = re.sub(r'\s+[a-zA-Z]\s+', ' ', lower_case)

    # Substituting multiple spaces with single space
    lower_case = re.sub(r'\s+', ' ', lower_case, flags=re.I)

    # Removing prefixed 'b'
    lower_case = re.sub(r'^b\s+', '', lower_case)

    # Removing non-english characters
    lower_case = re.sub(r'^b\s+', '', lower_case)

    # Return keywords which are not in stop words
    keywords = [word for word in words and word in punctuations and  word in w_num]

    return keywords

In [7]:
def clean_text_basic(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Removing stop words (example using NLTK library)
    # Make sure to install NLTK using: !pip install nltk
    from nltk.corpus import stopwords
    stopwords_list = set(stopwords.words('english'))
    tokens = text.split()
    tokens = [token for token in tokens if token not in stopwords_list]
    text = ' '.join(tokens)

    return text

In [8]:
pdf_path = r"/content/drive/MyDrive/Q&A from PDF - BERT module/Short-stories-from-100-Selected-Storiespdf.pdf"
text = extract_text_from_pdf(pdf_path)

In [9]:
text = clean_text_basic(text)

In [10]:
print(text[10000:100000])

 many years generous mint watermelon crops longshot winners new orleans racetrack brilliant banquets given indiana kansas citizens compose north carolina society made south rather fad manhattan manicure lisp softly left forefinger reminds much gentlemans rich mond va oh certainly many lady work war know henry selected stories dixie played darkhaired young man sprang somewhere mosby guerrilla yell waved frantically softbrimmed hat strayed smoke dropped vacant chair table pulled cigarettes evening period reserve thawed one us mentioned three wrzburgers waiter darkhaired young man acknowledged inclusion order smile nod hastened ask question wanted try theory would mind telling began whether fist e rushmore coglan banged table jarred silence excuse said thats question never like hear asked matter man fair judge man postoffice address ive seen kentuckians hated whisky virginians werent descended pocahon tas indianians hadnt written novel mexicans didnt wear velvet trousers silver dollars se

In [11]:
# we may use this for other applications to clean the data, like resume short listings, named entity recognition etc,
###text = to_lower(text)
###text = clean_text(text)

In [12]:
#print(text[:1000]) The sentences will be converted to words.

Training the model with Q&A PDF or the available documents


I don't have any question and answers pdf with me so instead of calling the dataset from the transformer using a json file (squad Q & A file) to train the model, the squad data has more than 80k data so its quiet impossible to train the model with a generic system so training the module with top 15 question and answer. (we may use other files to train the module)

In [13]:
import json

In [14]:
# Load the SQuAD dataset JSON file
#Training data
with open(r"/content/drive/MyDrive/Q&A from PDF - BERT module/SQuAD  train data set/train-v1.1.json", 'r') as f:
    squad_data = json.load(f)

# Process the SQuAD data to extract question-answer pairs
training_data = []
for article in squad_data['data']:
    for paragraph in article['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            if qa['answers']:
                answer = qa['answers'][0]['text']
                training_data.append({'question': question, 'answer': answer, 'context': context})

# Save the processed training data in a new JSON file
with open('training_data.json', 'w') as f:
    json.dump(training_data, f, indent=4)

print("Training data preparation complete.")


Training data preparation complete.


In [15]:
training_data[:15]

[{'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
  'answer': 'Saint Bernadette Soubirous',
  'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'},
 {'question': 'What is in front of the Notre Dame Main Building?',
  'answer': 'a copper statue of Christ',
  'context': 'Architecturally, the school has a

In [16]:
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
import torch

In [17]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Prepare and tokenize your training data
# Format: [{"context": "...", "question": "...", "answer": "..."}, ...]
train_data = training_data[:15]  # Replace with your training data

# Train the model
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate
model.train()

epochs = 8  # Adjust the number of epochs as needed

for epoch in range(epochs):
    total_loss = 0
    for example in train_data:
        context = example['context']
        question = example['question']
        answer = example['answer']

        # Tokenize the context and question
        encoded_data = tokenizer(context, question, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

        # Find the positions of the answer in the context
        start_idx = context.find(answer)
        end_idx = start_idx + len(answer)

        if start_idx == -1:  # Handle cases where answer is not found
            continue

        # Prepare the inputs with the answer token positions
        inputs = {
            'input_ids': encoded_data['input_ids'],
            'attention_mask': encoded_data['attention_mask'],
            'start_positions': torch.tensor([start_idx]),
            'end_positions': torch.tensor([end_idx])
        }

        optimizer.zero_grad()
        outputs = model(**inputs)

        # Check if the loss is valid (not NaN or inf)
        loss = outputs.loss
        if loss != loss:
            continue  # Skip this batch if loss is NaN

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Save the trained model
model.save_pretrained("trained_qa_model")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 69.2297
Epoch 2, Loss: 63.3135
Epoch 3, Loss: 59.3704
Epoch 4, Loss: 55.8294
Epoch 5, Loss: 49.5588
Epoch 6, Loss: 45.2368
Epoch 7, Loss: 41.4041
Epoch 8, Loss: 36.7165


In [18]:
def get_answer(context, question, model_path="trained_qa_model", max_seq_length=512):
    # Load the trained model and tokenizer
    model = BertForQuestionAnswering.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  # Same tokenizer as training

    # Tokenize the input question and context
    encoded_data = tokenizer.encode_plus(
        question,
        context,
        max_length=max_seq_length,
        truncation='only_second',  # Choose 'only_first' or 'only_second' to handle overflow
        padding='max_length',
        return_tensors='pt',
    )

    # Get the model's predicted start and end positions
    outputs = model(**encoded_data)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    # Find the token positions with the maximum probabilities
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    # Retrieve the tokenized answer span
    answer_tokens = encoded_data['input_ids'][0][start_idx:end_idx+1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# Training results

we didn't train the module with relevant questions and answers, so the module won't perform as expected, we need more computational power to train the model with squad data or we can use our own data(q&a data) with some relevant question and answers for our need to respond with our framed questions.

In [19]:
question = 'which price?'
predicted_answer = get_answer(context, question)
print("Predicted Answer:", predicted_answer)   #will return blank space if nothing is predicted

Predicted Answer: .


In [20]:
question = 'who is the wife character in the story?'
predicted_answer = get_answer(context, question)
print("Predicted Answer:", predicted_answer)   #will return blank space if nothing is predicted

Predicted Answer: .


# Pre-trained modes that provide more accurate results are listed and executed below with the same input

In [21]:
from transformers import pipeline

nlp = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", tokenizer="distilbert-base-cased")

context = text
question = "who is the main character in this story?"

# pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: anna maggie


In [22]:
#calling the pre-trained module, the execution time entirely depends on the size and length of the pdf file uploaded

context = text
question = 'what is missing in the story'

# pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: treasure child girl


# Using a different pdf to verify the performance of the model

In [23]:
pdf_path = r"/content/drive/MyDrive/Q&A from PDF - BERT module/romeo-and-juliet.pdf" #For easy access using a variable here and calling the function.
text = extract_text_from_pdf(pdf_path)
text = to_lower(text)

In [24]:
#calling the pre-trained module, the execution time entirely depends on the size and length of the pdf file uploaded

context = text
question = "where romeo met juliet?"

# Use the pipeline to answer the question
result = nlp(question=question, context=context, padding="max_length", truncation=True)

print("Answer:", result['answer'])

Answer: grave


In [25]:
# UI interface

In [26]:
!pip install -q gradio

In [27]:
import gradio as gr

In [28]:
# Function to perform question answering
def chat(pdf_file, question):
    try:
        # Validate uploaded PDF file
        if os.path.splitext(pdf_file.name)[1].lower() != ".pdf":
            return "Please upload a PDF file."

        # Extract text from uploaded PDF
        pdf_text = extract_text_from_pdf(pdf_file.name)

        # Use the pipeline to answer the question
        nlp = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", tokenizer="distilbert-base-cased")
        result = nlp(question=question, context=pdf_text, padding="max_length", truncation=True)
        answer = result['answer']

        return answer
    except Exception as e:
        # Log the exception for debugging purposes
        print("An error occurred:", e)
        return "An error occurred. Please try again later."

# Define the input components for the interface
input_components = [
    gr.inputs.File(label="Upload PDF file", type="file"),
    gr.inputs.Textbox(label="Enter your question")
]

# Create the interface
interface = gr.Interface(fn=chat, inputs=input_components, outputs="text", live=True)

# Launch the interface
interface.launch(share=True)


<ipython-input-28-d8d5f12efd5e>:24: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.File(label="Upload PDF file", type="file"),
<ipython-input-28-d8d5f12efd5e>:24: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.File(label="Upload PDF file", type="file"),
<ipython-input-28-d8d5f12efd5e>:24: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  gr.inputs.File(label="Upload PDF file", type="file"),
<ipython-input-28-d8d5f12efd5e>:25: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Enter your question")
<ipython-input-28-d8d5f12efd5e>:25: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Enter yo

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8c6c2db067f81e55ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
